<a href="https://colab.research.google.com/github/vaniamv/final-project-edit/blob/main/Streaming_speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# autentication to gcloud with login

!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=pZVybIRievZ58QFUceuUOpnXsMNzp8&prompt=consent&token_usage=remote&access_type=offline&code_challenge=RDGowfwKuxPuksIHJ8pwJpb6--oHMtESh-WJzWYAe2A&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrv2Y_BkwLnZ9G6FOX_gvnGPl2e1ysukCg2Oje53Kpn0elIH7jNu567HoEFETnNyAg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [4]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-11 14:15:11--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M   136MB/s    in 0.2s    

2025-01-11 14:15:11 (136 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]



In [5]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')


In [6]:
# Try reading a small file or a few records to avoid long processing times.
try:
  df = spark.read.format("json").load("gs://edit-de-project-streaming-data/carris-vehicles/vehicles_1736500532.json")
  df.show(5)  # Display the first 5 rows
  print("Connection successful!")
except Exception as e:
  print(f"Connection failed: {e}")

Connection failed: [PATH_NOT_FOUND] Path does not exist: gs://edit-de-project-streaming-data/carris-vehicles/vehicles_1736500532.json.


In [ ]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")

In [ ]:
stream.isStreaming

True

In [ ]:


output_path="gs://edit-data-eng-project-group1/datalake/stream/vehicles_5"

# writeStreaming
query = (stream
        .writeStream
        #.queryName('table')
        .outputMode("append")
        #.foreachBatch(insert_vehicles)
        .option("path", output_path)
        .option('checkpointLocation', 'gs://edit-data-eng-project-group1/datalake/stream/vehicles_5/checkpoint')
        #.trigger(processingTime='5 seconds')
        .start()

        )

query.awaitTermination(60)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [ ]:
query.isActive

True

In [ ]:
query.stop()

In [7]:
# Define the path to the Parquet files
parquet_path = "gs://edit-data-eng-project-group1/datalake/stream/vehicles_3"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()

# Perform some analysis or transformations (example: count rows)
#row_count = parquet_df.count()
#print(f"Number of rows: {row_count}")

# Example transformation: filter rows based on a condition
#filtered_df = parquet_df.filter(parquet_df.speed > 50)

# Show the filtered data
#filtered_df.show()

+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+---------------------------------+
|bearing|block_id                      |current_status|id      |lat      |line_id|lon      |pattern_id|route_id|schedule_relationship|shift_id    |speed    |stop_id|timestamp          |trip_id                          |
+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+---------------------------------+
|90     |20250103-64410390-223440234560|INCOMING_AT   |44|12697|38.70689 |4704   |-8.962685|4704_0_1  |4704_0  |SCHEDULED            |223440234560|13.333333|100398 |2025-01-03 18:36:13|4704_0_1|1400|1815_AEMLZ         |
|67     |20250103-64410147-223240234560|IN_TRANSIT_TO |44|12655|38.684013|4701   |-8.952688|4701_0_2  |4701_0  |SCHEDULE

In [10]:
from pyspark.sql.functions import lag , col, coalesce
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import math

# Define a window specification
windowSpec = Window.partitionBy("id").orderBy("timestamp")

#select columns
transform = parquet_df.select('id', 'speed', 'timestamp','line_id','route_id','stop_id','lat', 'lon')

# Create a new column 'previous_value' using lag
transform = transform.withColumn("previous_lat", coalesce(lag("lat", 1).over(windowSpec), col('lat')))
transform = transform.withColumn("previous_lon", coalesce(lag("lon", 1).over(windowSpec), col('lon')))

df_stops = spark.read.option("header", "true").csv('gs://edit-data-eng-project-group1/LandingZone/GTFS/stops.txt')
df_stops = df_stops.select('stop_id','stop_lat','stop_lon')
df_stops = df_stops.withColumn("stop_lat", df_stops["stop_lat"].cast("float"))
df_stops = df_stops.withColumn("stop_lon", df_stops["stop_lon"].cast("float"))
# Show the first few rows
transform = transform.join(df_stops, on='stop_id', how='left')


def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Calculate distance
    distance = R * c

    return distance

# Register the UDF
distance_udf = udf(haversine_distance, FloatType())

transform = transform.withColumn("distance", distance_udf(transform["previous_lat"],transform["previous_lon"],transform["lat"],transform["lon"]))
transform = transform.withColumn("distance_to_stop", distance_udf(transform["lat"],transform["lon"],transform["stop_lat"],transform["stop_lon"]))

# cache in memory
#transform.cache()

transform.show()

+-------+-------+---------+-------------------+-------+--------+---------+---------+------------+------------+---------+---------+------------+----------------+
|stop_id|     id|    speed|          timestamp|line_id|route_id|      lat|      lon|previous_lat|previous_lon| stop_lat| stop_lon|    distance|distance_to_stop|
+-------+-------+---------+-------------------+-------+--------+---------+---------+------------+------------+---------+---------+------------+----------------+
| 030785|41|1100|     15.0|2025-01-03 18:35:39|   1716|  1716_0| 38.72486|  -9.1829|    38.72486|     -9.1829|38.728104|-9.216909|         0.0|       2.9721527|
| 030785|41|1100|11.944445|2025-01-03 18:36:31|   1716|  1716_0| 38.72566|-9.191012|    38.72486|     -9.1829|38.728104|-9.216909|   0.7092681|       2.2628906|
| 030785|41|1100|15.833333|2025-01-03 18:36:49|   1716|  1716_0|38.725082|-9.193826|    38.72566|   -9.191012|38.728104|-9.216909|   0.2523195|       2.0304399|
| 030785|41|1100|4.4444447|2025-01

In [11]:
transform.printSchema()

root
 |-- stop_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- speed: float (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- line_id: string (nullable = true)
 |-- route_id: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- previous_lat: float (nullable = true)
 |-- previous_lon: float (nullable = true)
 |-- stop_lat: float (nullable = true)
 |-- stop_lon: float (nullable = true)
 |-- distance: float (nullable = true)
 |-- distance_to_stop: float (nullable = true)



In [61]:
transform.show()

+-------+-------+---------+-------------------+---------+---------+------------+------------+---------+---------+------------+----------------+
|stop_id|     id|    speed|          timestamp|      lat|      lon|previous_lat|previous_lon| stop_lat| stop_lon|    distance|distance_to_stop|
+-------+-------+---------+-------------------+---------+---------+------------+------------+---------+---------+------------+----------------+
| 030785|41|1100|     15.0|2025-01-03 18:35:39| 38.72486|  -9.1829|    38.72486|     -9.1829|38.728104|-9.216909|         0.0|       2.9721527|
| 030785|41|1100|11.944445|2025-01-03 18:36:31| 38.72566|-9.191012|    38.72486|     -9.1829|38.728104|-9.216909|   0.7092681|       2.2628906|
| 030785|41|1100|15.833333|2025-01-03 18:36:49|38.725082|-9.193826|    38.72566|   -9.191012|38.728104|-9.216909|   0.2523195|       2.0304399|
| 030785|41|1100|4.4444447|2025-01-03 18:37:27| 38.72312|-9.199276|   38.725082|   -9.193826|38.728104|-9.216909|   0.5206602|       1.6

In [17]:
import pyspark.sql.functions as F

agg = transform.groupBy("id", "stop_id", F.window("timestamp", "2 minutes")).agg(
    F.sum('distance').alias("distance_2_min"),
    F.min('distance').alias('distance_')
    )
agg = agg.withColumn('speed', col('distance_2_min')/(2/60))
agg.show()

+-------+-------+--------------------+--------------------+-----------+------------------+
|     id|stop_id|              window|      distance_2_min|  distance_|             speed|
+-------+-------+--------------------+--------------------+-----------+------------------+
|41|1104| 120083|{2025-01-03 19:26...|0.051517192274332047|        0.0|1.5455157682299614|
|41|1109| 060041|{2025-01-03 20:52...|  0.4222240149974823|        0.0|12.666720449924469|
|41|1109| 120533|{2025-01-04 00:02...|  0.0678267776966095| 0.06782678| 2.034803330898285|
|41|1109| 121099|{2025-01-04 01:00...| 0.04669193550944328|        0.0|1.4007580652832985|
|41|1114| 120157|{2025-01-03 20:48...| 0.43597667291760445|        0.0|13.079300187528133|
|41|1114| 121059|{2025-01-03 20:52...|  0.4232720732688904|0.063474596|12.698162198066711|
|41|1114| 121298|{2025-01-03 21:14...|  0.7600761950016022| 0.20100084|22.802285850048065|
|41|1117| 120923|{2025-01-03 23:08...|  0.4317595735192299|0.035660215|12.952787205576897|